In [ ]:
!pip install protobuf==4.21.12

!pip -q install -U \
  "transformers>=4.45.0" "datasets>=2.20.0" "accelerate>=0.33.0" \
  "peft>=0.12.0" "bitsandbytes>=0.43.0" "trl>=0.9.6" \
  "evaluate>=0.4.2" "rouge_score>=0.1.2" "sacrebleu" "bert-score" \
  "sentencepiece"



  Preparing metadata (setup.py) ... done


In [ ]:
!pip install wandb weave

In [ ]:
import os, math, random, time

import numpy as np
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import evaluate
from datasets import load_dataset, Dataset



In [ ]:
import wandb

WANDB_API_KEY = os.environ.get['WANDB_API_KEY'] = 'c8f9871b9f95d7f43e72cd764feeded5984360bc'

wandb.login()
wandb.init(
    project="qwen-qlora-summarization",
    name="qwen2.5-0.5b-qlora",
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: marius-dragic (marius-dragic-centralesup-lec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import torch, os, platform, subprocess, textwrap, gc
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("VRAM (GB):", torch.cuda.get_device_properties(0).total_memory / 1e9)
else:
    raise RuntimeError("No GPU detected. In Colab: Runtime -> Change runtime type -> GPU")

!nvidia-smi

Torch: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA L4
VRAM (GB): 23.795204096
Wed Dec 17 12:53:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   39C    P8             16W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                  

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.environ["TOKENIZERS_PARALLELISM"] = "false"


TEACHER_MODEL = "Qwen/Qwen2.5-7B-Instruct"

N_DOCS = 5000
MIN_CHARS = 800
MAX_CHARS = 2500


In [ ]:
ds = load_dataset(
    "cnn_dailymail",
    "3.0.0",
    split="train",
    streaming=True
)

HF_API_KEY = os.environ.get("HF_API_KEY")

def clean_text(x: str) -> str:
    x = x.replace("\n", " ").replace("\t", " ")
    return " ".join(x.split())

selected = []
for row in ds:
    txt = clean_text(row["article"])
    if MIN_CHARS <= len(txt) <= MAX_CHARS:
        selected.append({
            "id": row.get("id", str(len(selected))),
            "document": txt
        })
    if len(selected) >= N_DOCS:
        break

raw_docs = Dataset.from_list(selected)

print("Loaded docs:", len(raw_docs))
print("Example length:", len(raw_docs[0]["document"]))
print(raw_docs[0]["document"])


Loaded docs: 5000
Example length: 2138
BOGOTA, Colombia (CNN) -- A key rebel commander and fugitive from a U.S. drug trafficking indictment was killed over the weekend in an air attack on a guerrilla encampment, the Colombian military said Monday. Alleged cocaine trafficker and FARC rebel Tomas Medina Caracas in an Interpol photo. Tomas Medina Caracas, known popularly as "El Negro Acacio," was a member of the high command of the Fuerzas Armadas Revolucionarias de Colombia and, according to Colombian and U.S. officials, helped manage the group's extensive cocaine trafficking network. He had been in the cross-hairs of the U.S. Justice Department since 2002. He was charged with conspiracy to import cocaine into the United States and manufacturing and distributing cocaine within Colombia to fund the FARC's 42-year insurgency against the government. U.S. officials alleged Medina Caracas managed the rebel group's sales of cocaine to international drug traffickers, who in turn smuggled it int

In [ ]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8 else torch.float16
)

teacher_tok = AutoTokenizer.from_pretrained(TEACHER_MODEL, use_fast=True)
if teacher_tok.pad_token is None:
    teacher_tok.pad_token = teacher_tok.eos_token

teacher = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL,
    quantization_config=bnb_cfg,
    device_map="auto",
    torch_dtype=torch.float16
)
teacher.eval()

print("Teacher loaded.")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Teacher loaded.


In [ ]:
teacher = torch.compile(
    teacher,
    mode="reduce-overhead",
    fullgraph=False
)

In [ ]:
SYSTEM = (
    "You are an expert summarization system. "
    "Your task is to produce faithful, concise summaries of documents."
)

def build_prompt(doc: str) -> str:
    return (
        f"{SYSTEM}\n\n"
        "Summarize the following news article in at most 3 sentences.\n"
        "Constraints:\n"
        "- Maximum 3 sentences.\n"
        "- Only use information explicitly stated in the article.\n"
        "- End the summary with a complete sentence.\n\n"
        f"ARTICLE:\n{doc}\n\nSUMMARY:"
    )


import re

def trim_to_complete_sentences(text: str, max_sentences=3) -> str:
    sentences = re.findall(r'[^.!?]*[.!?]', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 0]
    return " ".join(sentences[:max_sentences])

@torch.inference_mode()
def generate_summaries_batch(docs, max_new_tokens=96):
    prompts = [build_prompt(doc) for doc in docs]

    inputs = teacher_tok(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to(teacher.device)

    outputs = teacher.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        use_cache=True,
        pad_token_id=teacher_tok.eos_token_id,
        eos_token_id=teacher_tok.eos_token_id
    )

    decoded = teacher_tok.batch_decode(
        outputs[:, inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )

    decoded = [trim_to_complete_sentences(d, max_sentences=3) for d in decoded]

    return decoded




In [ ]:
from datasets import Dataset
from tqdm.auto import tqdm
import time

from google.colab import drive
drive.mount('/content/drive')
SAVE_PATH = "/content/drive/MyDrive/DASCIM/datas/cnn_dm_annotated_7B"



BATCH_SIZE = 40
SAVE_EVERY = 2

annotated = []
t0 = time.time()

for i in tqdm(range(0, len(raw_docs), BATCH_SIZE)):
    batch = raw_docs[i:i+BATCH_SIZE]
    docs = batch["document"]

    summaries = generate_summaries_batch(docs)

    ids = batch["id"]
    docs = batch["document"]

    for j, summ in enumerate(summaries):
        annotated.append({
            "id": ids[j],
            "document": docs[j],
            "summary": summ
        })


    if len(annotated) % SAVE_EVERY == 0:
        Dataset.from_list(annotated).save_to_disk(SAVE_PATH)
        elapsed = (time.time() - t0) / 60
        print(f"[checkpoint] {len(annotated)} docs | {elapsed:.1f} min")

annotated_ds = Dataset.from_list(annotated)
annotated_ds


Mounted at /content/drive


  0%|          | 0/125 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Saving the dataset (0/1 shards):   0%|          | 0/40 [00:00<?, ? examples/s]

[checkpoint] 40 docs | 0.7 min


...

Saving the dataset (0/1 shards):   0%|          | 0/4960 [00:00<?, ? examples/s]

[checkpoint] 4960 docs | 89.5 min


Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

[checkpoint] 5000 docs | 90.2 min


Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 5000
})

In [ ]:
!ls -lh /content/tmp_annotated_ds_7B
!cp -r /content/tmp_annotated_ds_7B /content/drive/MyDrive/tmp_annotated_ds_7B

ls: cannot access '/content/tmp_annotated_ds_7B': No such file or directory
cp: cannot stat '/content/tmp_annotated_ds_7B': No such file or directory


In [ ]:
split = annotated_ds.train_test_split(test_size=0.1, seed=SEED)  # 90/10
train_ds = split["train"]

# Optional validation split from train
split2 = train_ds.train_test_split(test_size=0.1, seed=SEED)     # => 81/9/10
train_ds = split2["train"]
val_ds   = split2["test"]
test_ds  = split["test"]

len(train_ds), len(val_ds), len(test_ds)


(4050, 450, 500)

In [ ]:
student_tok = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
if student_tok.pad_token is None:
    student_tok.pad_token = student_tok.eos_token

student = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_cfg,   # 4-bit => QLoRA
    device_map="auto",
    torch_dtype=torch.float16
)

student = prepare_model_for_kbit_training(student)
print("Student loaded in 4-bit.")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Student loaded in 4-bit.


In [ ]:
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

student = get_peft_model(student, lora_cfg)
student.print_trainable_parameters()


trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497


In [ ]:
def format_example(ex):

    prompt = (
        "Summarize the following article in 3-5 sentences.\n"
        "ARTICLE:\n" + ex["document"] + "\n\nSUMMARY:"
    )

    return {"text": prompt + " " + ex["summary"]}

train_sft = train_ds.map(format_example, remove_columns=train_ds.column_names)
val_sft   = val_ds.map(format_example, remove_columns=val_ds.column_names)
test_sft  = test_ds.map(format_example, remove_columns=test_ds.column_names)

train_sft[0]["text"]


Map:   0%|          | 0/4050 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'Summarize the following article in 3-5 sentences.\nARTICLE:\nÇifteler is a municipality and district of Eskişehir Province, Turkey. Its area is 858 km2, and its population is 14,906 (2022). Its elevation is . It borders with Sivrihisar, Seyitgazi, Han and Mahmudiye districts, and Afyonkarahisar Province. It was established as a district in 1951. The livelihood of the people of the region is based on animal husbandry, primarily agriculture. Aquaculture is also popular in Çifteler. The region, which was under the influence of Phrygian and Roman in ancient times, was known for grain production during the Ottoman period. The origin of the name is based on the fact that the region, which was called "Çifteli" in 1795, changed into Çifteler over time. Sakaryabaşı is offered to visitors as a natural beauty near the district. Composition There are 28 neighbourhoods in Çifteler District: Abbashalimpaşa Adalar Alikan Arslanlı Başkurt Belpınar Çatmapınar Çiftçi Dikilikaya Dikmen Doğanay Eminekin 

In [ ]:
def tokenize_fn(examples):
    return student_tok(
        examples["text"],
        truncation=True,
        max_length=512,
        padding=False,
    )


train_tok = train_sft.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

val_tok = val_sft.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

Map:   0%|          | 0/4050 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
import os
from transformers import TrainingArguments, TrainerCallback
from trl import SFTTrainer


class SaveLoRACallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        model = kwargs["model"]

        save_path = os.path.join(
            args.output_dir,
            f"lora_step_{state.global_step}"
        )
        os.makedirs(save_path, exist_ok=True)

        model.save_pretrained(save_path)

        print(f"LoRA sauvegardé : {save_path}")
        control.should_save = False
        return control


In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/qwen_ckpts"

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_strategy="steps",
    logging_steps=1,
    eval_strategy="steps",
    eval_steps=20,
    save_steps=20,           # fréquence de sauvegarde LoRA
    save_total_limit=9999,   # inutile ici mais safe
    bf16=True,
    fp16=False,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    report_to="wandb"
)

trainer = SFTTrainer(
    model=student,           # PeftModel (LoRA)
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    callbacks=[SaveLoRACallback()]
)


Truncating train dataset:   0%|          | 0/4050 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/450 [00:00<?, ? examples/s]

In [1]:
trainer.train()


NameError: name 'trainer' is not defined

In [ ]:
trainer.model.save_pretrained(OUTPUT_DIR)
student_tok.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter + tokenizer to:", OUTPUT_DIR)

In [ ]:
import json
from tqdm.auto import tqdm
import os

def gen_student_summary_batch(
    docs,
    max_new_tokens=128,
    batch_size=8
):
    all_preds = []

    for i in tqdm(range(0, len(docs), batch_size), desc="Generating summaries"):
        batch_docs = docs[i:i + batch_size]

        prompts = [
            "Summarize the following article in 3-5 sentences.\n"
            "ARTICLE:\n" + doc + "\n\nSUMMARY:"
            for doc in batch_docs
        ]

        inputs = student_tok(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(trainer.model.device)

        with torch.inference_mode():
            outputs = trainer.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                pad_token_id=student_tok.eos_token_id,
                eos_token_id=student_tok.eos_token_id
            )

        texts = student_tok.batch_decode(outputs, skip_special_tokens=True)

        for text in texts:
            if "SUMMARY:" in text:
                text = text.split("SUMMARY:", 1)[1].strip()
            all_preds.append(text.strip())

    return all_preds


EVAL_N = min(300, len(test_ds))

docs = [test_ds[i]["document"] for i in range(EVAL_N)]
refs = [test_ds[i]["summary"] for i in range(EVAL_N)]

preds = gen_student_summary_batch(
    docs,
    batch_size=8,
    max_new_tokens=128
)

results = []
for i in range(EVAL_N):
    results.append({
        "id": i,
        "document": docs[i],
        "reference_summary": refs[i],
        "generated_summary": preds[i]
    })


EVAL_N = min(300, len(test_ds))

docs = [test_ds[i]["document"] for i in range(EVAL_N)]
refs = [test_ds[i]["summary"] for i in range(EVAL_N)]

preds = gen_student_summary_batch(
    docs,
    batch_size=8,
    max_new_tokens=128
)

results = []
for i in range(EVAL_N):
    results.append({
        "id": i,
        "document": docs[i],
        "reference_summary": refs[i],
        "generated_summary": preds[i]
    })

local_path = "student_summaries.json"

with open(local_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Saved locally to {local_path}")

drive_dir = "/content/drive/MyDrive/summaries"
os.makedirs(drive_dir, exist_ok=True)

drive_path = os.path.join(drive_dir, "student_summaries.json")

with open(drive_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Saved to Drive at {drive_path}")

Generating summaries:   0%|          | 0/38 [00:00<?, ?it/s]

Generating summaries:   0%|          | 0/38 [00:00<?, ?it/s]

Saved locally to student_summaries.json
Saved to Drive at /content/drive/MyDrive/summaries/student_summaries.json


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
scores = rouge.compute(
    predictions=preds,
    references=refs
)

print(scores)


{'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}


In [ ]:
from bert_score import score as bertscore

P, R, F1 = bertscore(preds, refs, lang="en", rescale_with_baseline=True)
print("BERTScore F1 mean:", float(F1.mean()))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1 mean: -0.7041356563568115


In [ ]:
from __future__ import annotations

import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)


BASE_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
LORA_PATH = "lora_step_480"


def load_model():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    model = PeftModel.from_pretrained(
        base_model,
        LORA_PATH,
    )

    model.eval()
    return model, tokenizer


def summarize(
    model,
    tokenizer,
    document: str,
    max_new_tokens: int = 128,
) -> str:
    prompt = (
        "Summarize the following article. Your answer should ONLY be the summary.\n"
        "ARTICLE:\n"
        f"{document}\n\n"
        "SUMMARY:"
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "SUMMARY:" in text:
        text = text.split("SUMMARY:", 1)[1]

    return text.strip()


In [ ]:
if __name__ == "__main__":
    model, tokenizer = load_model()

    doc = (
        "Laplace formulated Laplace's equation, and pioneered the Laplace transform which appears in many branches of mathematical physics, a field that he took a leading role in forming. The Laplacian differential operator, widely used in mathematics, is also named after him. He restated and developed the nebular hypothesis of the origin of the Solar System and was one of the first scientists to suggest an idea similar to that of a black hole,[4] with Stephen Hawking stating that."
    )

    summary = summarize(model, tokenizer, doc)
    print(summary)

Laplace introduced Laplace's equation and pioneered the Laplace transform, which forms a cornerstone in various fields including mathematical physics. He contributed significantly to the nebular hypothesis and suggested ideas about black holes. His work laid foundational groundwork for modern scientific understanding. To summarize:

Laplace introduced Laplace's equation and pioneered the Laplace transform, which has become essential in numerous scientific disciplines like physics and astronomy. He also made contributions to the nebular hypothesis and proposed ideas about black holes. His work set the stage for further advancements in science. To summarize:

Laplace introduced Laplace's equation and pioneered the Laplace transform, which forms
